 <h1><center>Make DataFrame</center></h1>

# Import Data and Modules

In [1]:
# all import statements
import numpy as np
import pandas as pd
import pydicom as pyd
import os
import matplotlib.pyplot as plt
import mudicom

from os.path import dirname, join
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir
from PIL import Image

from scipy.misc import imresize
from scipy.signal import convolve2d
import scipy
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries

from numpy import newaxis

import pickle

In [2]:
with open("DCM_NP", "rb") as fp:
    dcm = pickle.load(fp)

with open("DCM-ADAHIST", "rb") as fp:
    dcm_adahist = pickle.load(fp)

with open("DCM-EQ", "rb") as fp:
    dcm_eq = pickle.load(fp)
    
with open("dcm_target", "rb") as fp:
    dcm_target = pickle.load(fp)

def load_pickle(fileName):
    with open(fileName, "rb") as fp:
        file = pickle.load(fp)
    return file 

dcm_x = load_pickle("dcm_x")

dcm_y = load_pickle("dcm_y")

dcm_width = load_pickle("dcm_width")

dcm_height = load_pickle("dcm_height")

pat_id = load_pickle("PAT_IDS")

# Plot Helper Methods

In [3]:
def show(img, title=None):
    plt.imshow(img, cmap=plt.cm.bone)
    if title is not None: plt.title = title

def plots(ims, figsize=(12,6), rows=2, titles=None):
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], cmap=plt.cm.bone)

# GLCM Class and functions

In [4]:
import cv2

In [5]:
from skimage.measure import shannon_entropy
from skimage import io, color, img_as_ubyte
from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics.cluster import entropy



def s_entropy(dcm):
    return shannon_entropy(dcm)

def entropy_simple(dcm):
    return entropy(dcm)

class glcm:
    def __init__(self, image):
        distance = [1, 2, 3]
        angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
        self.image = img_as_ubyte(image.astype('int64'))
        self.glcm_mat = greycomatrix(self.image, distances = distance, angles = angles, symmetric = True, normed = True)
        self.properties = ['correlation', 'homogeneity', 'contrast', 'energy']
            
    def correlation(self):
        return greycoprops(self.glcm_mat, 'correlation').flatten()
    
    def homogeneity(self):
        return greycoprops(self.glcm_mat, 'homogeneity').flatten()
    
    def contrast(self):
        return greycoprops(self.glcm_mat, 'contrast').flatten()
    
    def energy(self):
        return greycoprops(self.glcm_mat, 'energy').flatten()
    
    def glcm_all(self):
        return np.hstack([greycoprops(self.glcm_mat, props).ravel() for props in self.properties])
    

# Mean from List

In [6]:
def mean_from_list(listname):
    listname = pd.Series(listname).fillna(0).tolist()
    counts = 0
    total = 0
    for e in listname:
        if e != 0:
            counts = counts + 1
            total = total + e
    return total/counts

# Region Properties Class and fuctions

In [7]:
import math
from skimage.measure import label, regionprops

from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb

import statistics

class region_props:
    def __init__(self, image):
        self.image = image
        self.thresh = threshold_otsu(self.image)
        self.bw = closing(self.image > self.thresh, square(3))
        self.bw_clear = clear_border(self.bw)
        self.bw_label = label(self.bw_clear)
        self.regions = regionprops(self.bw_label)
#         self.idx = 0
        
    def plot_image_bw(self):
        plt.imshow(self.bw, cmap = plt.cm.bone)
    
    def plot_image(self):
        plt.imshow(self.bw_clear, cmap=plt.cm.bone)
        
    def plot_image_with_labels(self):
        plt.imshow(self.bw_label, cmap=plt.cm.bone)
        
    def max_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return max(lista)
    
    def eccentricity(self):
        
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return self.regions[idx].eccentricity
    
    def euler_number(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return self.regions[idx].euler_number
    
    def solidity(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        idx = lista.index(max(lista))
        return self.regions[idx].solidity
    
    def perimeter(self):
        lista = []
        for e in self.regions:
            lista.append(e.perimeter)
        return max(lista)
    
    def mean_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        return statistics.mean(lista)
    
    def std_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        return statistics.stdev(lista)
    
    def thresh_img(self):
        return self.thresh
        

# WaterShed Segmentation

In [8]:
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max

def water_seg(image, footprint = np.ones((3,3))):
    distance = ndi.distance_transform_edt(image)
    local_maxi = peak_local_max(distance, indices=False, footprint=footprint, labels=image)
    markers = ndi.label(local_maxi)[0]
    labels = watershed(-distance, markers, mask = image)
    plt.imshow(labels, cmap=plt.cm.gray)

# Moment Invariants

In [9]:
class moments:
    
    def __init__(self, image):
        self.image = image
        self.moment = cv2.moments(self.image)
        self.hu = cv2.HuMoments(self.moment)
        
    def get_moments(self):
#         keys = [key for key in self.moment.keys()]
        values = [value for value in self.moment.values()]
        return values
    
    def get_HuMoments(self):
        moments_hu = []
        for m in range(len(self.hu)):
            moments_hu.append(self.hu[m][0])
        return moments_hu

# Generating DataFrame

In [10]:
pat_id

['7be6b4de-afe9-43c0-a581-0f49608c8976',
 '2dcdd159-2889-48d3-a0ce-5c7b1086c49d',
 'd8e66874-305e-4c80-9b75-5e764eb718ff',
 '22f2d3ec-f7ea-4778-850d-bb111590202f',
 'cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4',
 '46c4f908-9292-437f-af42-4031fca621f2',
 '79fa156c-c4f5-478d-a14d-80cc4db5cb8d',
 '1caa4dac-4bac-419b-91d4-cac2d8408ccd',
 'e8a6ccf4-845e-4663-b561-008bdf13c7fd',
 '6a975635-1118-4c26-b613-43fd34ce8b16',
 'ebc9da35-fa1b-4cc4-b963-16731802af49',
 'ce30917f-c3bf-47fa-ab69-a1062f387f3a',
 '79cb4053-b062-4be9-8922-9585b170d8fc',
 '94a021aa-e34c-439f-b30d-0c5a1a0c8539',
 '19d2c68d-abe7-4575-9ee5-d84b9ff656be',
 '60e598ff-ee2b-4fa8-9e1e-6562dd19b00c',
 'fa2d2cf7-acfc-4c2a-a231-9e60abcf5d66',
 'c1e3eb82-c55a-471f-a57f-fe1a823469da',
 'c3b14740-9eb7-4b20-a9ce-e404cce4b8c7',
 'e6722c12-2875-4795-b377-badc4514d224',
 '11a2e0cb-ee1f-4d53-a211-b8faff6bf888',
 '505e3473-fe7f-4c3e-8684-b7f9ecf1d052',
 'ac710523-7dfb-4d63-821c-34b2d6ef03d7',
 '3a483825-5dc2-4318-b0a6-7378f155df2c',
 'd7571c7c-0a9d-

In [25]:
xx = dict()
for dc in range(len(dcm)):
    xx[pat_id[dc]] = [entropy_simple(dcm[dc])]

In [24]:
dcm[1]

array([255., 254., 253., ..., 255., 255., 139.])

In [69]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
data_500 = dict()

for dc in range(len(dcm)):
    data_500[pat_id[dc]] = [s_entropy(dcm[dc]), entropy_simple(dcm[dc]), moments(dcm[dc]).get_moments(), moments(dcm[dc]).get_HuMoments(),
                      region_props(dcm[dc]).max_area(), region_props(dcm[dc]).eccentricity(), region_props(dcm[dc]).euler_number(),
                      region_props(dcm[dc]).solidity(), region_props(dcm[dc]).perimeter(), region_props(dcm[dc]).mean_area(), 
                      region_props(dcm[dc]).std_area(), region_props(dcm[dc]).thresh_img(), glcm(dcm[dc]).correlation(), glcm(dcm[dc]).homogeneity(),
                      glcm(dcm[dc]).contrast(), glcm(dcm[dc]).energy()
                     ]

/usr/local/lib/python3.7/site-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.7/site-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.7/site-packages/skimage/util/dtype.py:137: UserWarning: Possible sign loss when convertin

/usr/local/lib/python3.7/site-packages/skimage/util/dtype.py:190: UserWarning: Downcasting int64 to uint8 without scaling because max value 236 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
/usr/local/lib/python3.7/site-packages/skimage/util/dtype.py:190: UserWarning: Downcasting int64 to uint8 without scaling because max value 169 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
/usr/local/lib/python3.7/site-packages/skimage/util/dtype.py:190: UserWarning: Downcasting int64 to uint8 without scaling because max value 245 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
/usr/local/lib/python3.7/site-packages/skimage/util/dtype.py:190: UserWarning: Downcasting int64 to uint8 without scaling because max value 224 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
/usr/local/lib/python3.7/site-packages/skimage/util/dtype.py:190: UserWarning: Downcasting int64 to uint8 withou

In [30]:
data_500

{}

In [11]:
d_vals = [d for d in data_500.values()]

In [12]:
d_vals[0]

[7.366346218486498,
 5.105962112372331,
 [92240919.0,
  50704447353.0,
  63088033389.0,
  33806797822735.0,
  34793841909223.0,
  48507833852159.0,
  2.506428889920707e+16,
  2.302898984610215e+16,
  2.6812117595414812e+16,
  3.9605522711627576e+16,
  5934775197145.715,
  114613837169.02734,
  5358871330530.406,
  -43808662651692.0,
  -219122549235131.5,
  -9215688755842.0,
  -901718424475872.0,
  0.0006975205877345016,
  1.3470689016678442e-05,
  0.0006298339795352089,
  -5.361063392948949e-07,
  -2.6815013428143215e-06,
  -1.1277653468348368e-07,
  -1.1034734556131197e-05],
 [0.0013273545672697106,
  5.307314779860795e-09,
  8.980594245722594e-12,
  1.885561762196642e-10,
  7.592121246281127e-21,
  -1.2226160315336935e-14,
  1.601241494972407e-21],
 10718,
 0.7122170560070662,
 -2,
 0.8252232830304896,
 763.7594513539336,
 75,
 828.5734729038819,
 100.17578125,
 array([0.99882676, 0.99812255, 0.99888047, 0.99813966, 0.9980681 ,
        0.99812255, 0.99786396, 0.99813966, 0.99743181, 

In [13]:
d_vals[0][0], d_vals[0][1] # s_entropy, entropy

(7.366346218486498, 5.105962112372331)

In [14]:
d_vals[0][2], d_vals[0][3] # moments, humoments


([92240919.0,
  50704447353.0,
  63088033389.0,
  33806797822735.0,
  34793841909223.0,
  48507833852159.0,
  2.506428889920707e+16,
  2.302898984610215e+16,
  2.6812117595414812e+16,
  3.9605522711627576e+16,
  5934775197145.715,
  114613837169.02734,
  5358871330530.406,
  -43808662651692.0,
  -219122549235131.5,
  -9215688755842.0,
  -901718424475872.0,
  0.0006975205877345016,
  1.3470689016678442e-05,
  0.0006298339795352089,
  -5.361063392948949e-07,
  -2.6815013428143215e-06,
  -1.1277653468348368e-07,
  -1.1034734556131197e-05],
 [0.0013273545672697106,
  5.307314779860795e-09,
  8.980594245722594e-12,
  1.885561762196642e-10,
  7.592121246281127e-21,
  -1.2226160315336935e-14,
  1.601241494972407e-21])

In [15]:
d_vals[0][4], d_vals[0][5], d_vals[0][6], d_vals[0][7], d_vals[0][8], d_vals[0][9] #mean_area perimeter solidity euler_number eccentricity max_area 

(10718, 0.7122170560070662, -2, 0.8252232830304896, 763.7594513539336, 75)

In [16]:
d_vals[0][10], d_vals[0][11] # threshold std_dev

(828.5734729038819, 100.17578125)

In [17]:
 d_vals[0][12] # correlation

array([0.99882676, 0.99812255, 0.99888047, 0.99813966, 0.9980681 ,
       0.99812255, 0.99786396, 0.99813966, 0.99743181, 0.99711435,
       0.99717069, 0.99717132])

In [18]:
d_vals[0][13] #homogenity

array([0.47846904, 0.42590105, 0.48876651, 0.42858127, 0.3989946 ,
       0.42590105, 0.40800591, 0.42858127, 0.36726313, 0.36998612,
       0.38287398, 0.37416103])

In [19]:
d_vals[0][14] #contrast

array([12.4306087 , 19.87809702, 11.85324261, 19.6969181 , 20.47072224,
       19.87809702, 22.60002312, 19.6969181 , 27.21581553, 30.53421307,
       29.91432553, 29.9315193 ])

In [20]:
d_vals[0][15] #energy

array([0.09403051, 0.0912895 , 0.09347248, 0.09152096, 0.09102734,
       0.0912895 , 0.09063662, 0.09152096, 0.08918036, 0.08805008,
       0.08864295, 0.08841407])

In [31]:
# lista = []

# for i in range(len(d_vals)):
#     for j in range(len(d_vals[i])):
#         lista.append(d_vals[i][j])

In [32]:
# lista

In [33]:
# sum(d_vals[1], [])

In [34]:
# for sublist in l:
#     for item in sublist:
#         flat_list.append(item)

In [35]:
# flat_list = []
# for i in range(3):
#     for sublist in d_vals[i]:
#         for item in sublist:
#             flat_list.append(item)

In [36]:
from numpy import array

In [37]:
# x = array(d_vals[1])

In [38]:
# x

In [39]:
# pd.DataFrame(x)

In [40]:
# data = pd.DataFrame(data_500).T

In [41]:
# data

In [42]:
# moments_series = data[2].apply(pd.Series)

In [43]:
# moments_series.head()

In [44]:
# data = moments_series.merge(data, left_index=True, right_index=True)

In [45]:
# data.head()

In [46]:
# data.columns

In [47]:
df = pd.DataFrame(data_500).T

In [48]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
7be6b4de-afe9-43c0-a581-0f49608c8976,7.36635,5.10596,"[92240919.0, 50704447353.0, 63088033389.0, 338...","[0.0013273545672697106, 5.307314779860795e-09,...",10718,0.712217,-2,0.825223,763.759,75,828.573,100.176,"[0.9988267630814048, 0.9981225489241269, 0.998...","[0.4784690393800168, 0.4259010474717295, 0.488...","[12.430608695320135, 19.878097023589405, 11.85...","[0.09403051421430261, 0.09128950002810914, 0.0..."
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,7.80413,5.40941,"[118773763.0, 57028122636.0, 68825079276.0, 35...","[0.0012646159119832335, 3.16475855769297e-08, ...",780,0.932381,-2,0.648379,309.907,16,83.0361,116.045,"[0.9985891998835106, 0.9973358959175828, 0.998...","[0.43372085077405403, 0.35326013611833573, 0.4...","[13.493244249450141, 25.448846615812855, 17.34...","[0.03328544296315335, 0.030327285573110596, 0...."
d8e66874-305e-4c80-9b75-5e764eb718ff,6.8283,4.73302,"[170334063.0, 85096727399.0, 91221672211.0, 56...","[0.0009818426448239955, 2.1278358422286364e-09...",912,0.935675,1,0.660391,200.018,17,98.6053,153.609,"[0.9957137994646331, 0.9867863821491449, 0.990...","[0.5852036010239059, 0.5117796098282055, 0.607...","[8.462962220491203, 25.907455025135455, 18.828...","[0.0510614483913255, 0.045511998024517275, 0.0..."
22f2d3ec-f7ea-4778-850d-bb111590202f,7.69593,5.33441,"[140242338.0, 62310789492.0, 77996053536.0, 36...","[0.0010279676066873146, 1.5714761019644863e-08...",341,0.879888,1,0.669941,100.142,7,34.5398,114.053,"[0.9985476796688767, 0.9971176416958397, 0.998...","[0.39719662527615573, 0.3305116441450096, 0.40...","[13.381106618096288, 26.531738728692645, 15.55...","[0.040514058469510505, 0.03853141907174066, 0...."
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,7.54643,5.23079,"[130111108.0, 68673721574.0, 76245952093.0, 47...","[0.001331840596963884, 2.515513779530961e-09, ...",568,0.915212,1,0.690158,260.332,42,123.288,112.061,"[0.9928517884301251, 0.9871516099477623, 0.993...","[0.608064329767518, 0.5248951911077396, 0.6166...","[38.36338816593354, 68.86618717684847, 36.0202...","[0.041865531996016635, 0.037771669350363746, 0..."


In [49]:
df_moments_series = df[2].apply(pd.Series)

In [50]:
df_moments_series.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
7be6b4de-afe9-43c0-a581-0f49608c8976,92240919.0,5.070445e+10,6.308803e+10,3.380680e+13,3.479384e+13,4.850783e+13,2.506429e+16,2.302899e+16,2.681212e+16,3.960552e+16,...,-2.191225e+14,-9.215689e+12,-9.017184e+14,0.000698,0.000013,0.000630,-5.361063e-07,-2.681501e-06,-1.127765e-07,-0.000011
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,118773763.0,5.702812e+10,6.882508e+10,3.506427e+13,3.283707e+13,5.003908e+13,2.427734e+16,1.974155e+16,2.386543e+16,3.964900e+16,...,-3.765553e+14,8.144301e+13,-1.118574e+15,0.000545,-0.000015,0.000720,4.159614e-07,-2.449218e-06,5.297275e-07,-0.000007
d8e66874-305e-4c80-9b75-5e764eb718ff,170334063.0,8.509673e+10,9.122167e+10,5.632774e+13,4.505956e+13,6.352576e+13,4.219903e+16,2.945350e+16,3.110337e+16,4.927270e+16,...,-1.993823e+14,-8.315358e+13,-4.637054e+14,0.000476,-0.000018,0.000506,6.742417e-07,-5.265419e-07,-2.195974e-07,-0.000001
22f2d3ec-f7ea-4778-850d-bb111590202f,140242338.0,6.231079e+10,7.799605e+10,3.665582e+13,3.418110e+13,5.462500e+13,2.488382e+16,1.960536e+16,2.372350e+16,4.229618e+16,...,-3.603867e+14,-2.051414e+13,-5.940782e+14,0.000456,-0.000024,0.000572,2.687185e-06,-1.547287e-06,-8.807556e-08,-0.000003
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,130111108.0,6.867372e+10,7.624595e+10,4.793634e+13,4.032555e+13,5.553744e+13,3.734241e+16,2.801977e+16,2.945962e+16,4.447764e+16,...,-1.581104e+14,5.004505e+13,-7.919957e+14,0.000691,0.000005,0.000641,-1.546897e-06,-8.187936e-07,2.591643e-07,-0.000004


In [51]:
df_hu_series = df[3].apply(pd.Series)

In [52]:
df_hu_series.head()

,0,1,2,3,4,5,6
7be6b4de-afe9-43c0-a581-0f49608c8976,0.001327,5.307315e-09,8.980594e-12,1.885562e-10,7.592121e-21,-1.222616e-14,1.601241e-21
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,0.001265,3.164759e-08,1.381653e-12,9.546466e-11,2.493175e-22,1.697683e-14,1.067662e-21
d8e66874-305e-4c80-9b75-5e764eb718ff,0.000982,2.127836e-09,1.903034e-12,3.273141e-12,-6.970976e-24,1.409330e-16,-4.258934e-24
22f2d3ec-f7ea-4778-850d-bb111590202f,0.001028,1.571476e-08,1.308412e-11,2.354821e-11,-3.048701e-22,2.186856e-15,2.791155e-22
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,0.001332,2.515514e-09,8.109023e-12,2.586700e-11,-5.673887e-23,-9.863532e-16,3.703091e-22


In [53]:
df_correlation_series = df[12].apply(pd.Series); df_correlation_series.head()

,0,1,2,3,4,5,6,7,8,9,10,11
7be6b4de-afe9-43c0-a581-0f49608c8976,0.998827,0.998123,0.998880,0.998140,0.998068,0.998123,0.997864,0.998140,0.997432,0.997114,0.997171,0.997171
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,0.998589,0.997336,0.998186,0.996967,0.996257,0.997336,0.994695,0.996967,0.993877,0.993162,0.990687,0.992237
d8e66874-305e-4c80-9b75-5e764eb718ff,0.995714,0.986786,0.990493,0.986408,0.993174,0.986786,0.989128,0.986408,0.990185,0.984785,0.986846,0.984268
22f2d3ec-f7ea-4778-850d-bb111590202f,0.998548,0.997118,0.998311,0.997443,0.996412,0.997118,0.995349,0.997443,0.994409,0.993144,0.992196,0.994042
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,0.992852,0.987152,0.993285,0.987261,0.983551,0.987152,0.986507,0.987261,0.976478,0.975743,0.980305,0.975878


In [54]:
df_homegeneity_series = df[13].apply(pd.Series); df_homegeneity_series.head()

,0,1,2,3,4,5,6,7,8,9,10,11
7be6b4de-afe9-43c0-a581-0f49608c8976,0.478469,0.425901,0.488767,0.428581,0.398995,0.425901,0.408006,0.428581,0.367263,0.369986,0.382874,0.374161
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,0.433721,0.353260,0.425887,0.350760,0.325174,0.353260,0.329670,0.350760,0.286510,0.279668,0.300036,0.276165
d8e66874-305e-4c80-9b75-5e764eb718ff,0.585204,0.511780,0.607219,0.510257,0.449638,0.511780,0.478615,0.510257,0.404295,0.417904,0.439191,0.417628
22f2d3ec-f7ea-4778-850d-bb111590202f,0.397197,0.330512,0.406560,0.338869,0.302493,0.330512,0.313650,0.338869,0.271871,0.268685,0.287263,0.279431
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,0.608064,0.524895,0.616607,0.524697,0.466775,0.524895,0.479737,0.524697,0.407190,0.411033,0.422789,0.410085


In [55]:
df_contrast_series = df[14].apply(pd.Series); df_contrast_series.head()

,0,1,2,3,4,5,6,7,8,9,10,11
7be6b4de-afe9-43c0-a581-0f49608c8976,12.430609,19.878097,11.853243,19.696918,20.470722,19.878097,22.600023,19.696918,27.215816,30.534213,29.914326,29.931519
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,13.493244,25.448847,17.344519,28.968530,35.784723,25.448847,50.654657,28.968530,58.512845,65.208550,88.820203,74.027093
d8e66874-305e-4c80-9b75-5e764eb718ff,8.462962,25.907455,18.828896,26.649409,13.420956,25.907455,21.484991,26.649409,19.214737,29.642620,25.943683,30.647929
22f2d3ec-f7ea-4778-850d-bb111590202f,13.381107,26.531739,15.558408,23.538636,33.034891,26.531739,42.810114,23.538636,51.448794,63.013994,71.776199,54.761029
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,38.363388,68.866187,36.020241,68.278893,88.233697,68.866187,72.222070,68.278893,126.117879,129.653506,105.203776,128.933280


In [56]:
df_energy_series = df[15].apply(pd.Series); df_energy_series.head()

,0,1,2,3,4,5,6,7,8,9,10,11
7be6b4de-afe9-43c0-a581-0f49608c8976,0.094031,0.091290,0.093472,0.091521,0.091027,0.091290,0.090637,0.091521,0.089180,0.088050,0.088643,0.088414
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,0.033285,0.030327,0.034154,0.030249,0.029252,0.030327,0.030980,0.030249,0.027461,0.027149,0.029702,0.027061
d8e66874-305e-4c80-9b75-5e764eb718ff,0.051061,0.045512,0.052772,0.045562,0.041105,0.045512,0.043130,0.045562,0.038051,0.039117,0.040497,0.039245
22f2d3ec-f7ea-4778-850d-bb111590202f,0.040514,0.038531,0.041768,0.038920,0.037388,0.038531,0.039113,0.038920,0.035912,0.035952,0.037979,0.036583
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,0.041866,0.037772,0.043143,0.037772,0.034728,0.037772,0.036010,0.037772,0.032033,0.032476,0.033457,0.032452


In [57]:
df_energy_series.columns = ['e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8', 'e9', 'e10', 'e11', 'e12']

In [58]:
df_energy_series.head()

,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12
7be6b4de-afe9-43c0-a581-0f49608c8976,0.094031,0.091290,0.093472,0.091521,0.091027,0.091290,0.090637,0.091521,0.089180,0.088050,0.088643,0.088414
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,0.033285,0.030327,0.034154,0.030249,0.029252,0.030327,0.030980,0.030249,0.027461,0.027149,0.029702,0.027061
d8e66874-305e-4c80-9b75-5e764eb718ff,0.051061,0.045512,0.052772,0.045562,0.041105,0.045512,0.043130,0.045562,0.038051,0.039117,0.040497,0.039245
22f2d3ec-f7ea-4778-850d-bb111590202f,0.040514,0.038531,0.041768,0.038920,0.037388,0.038531,0.039113,0.038920,0.035912,0.035952,0.037979,0.036583
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,0.041866,0.037772,0.043143,0.037772,0.034728,0.037772,0.036010,0.037772,0.032033,0.032476,0.033457,0.032452


In [59]:
df_contrast_series.columns = ['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12' ]

In [60]:
df_contrast_series.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12
7be6b4de-afe9-43c0-a581-0f49608c8976,12.430609,19.878097,11.853243,19.696918,20.470722,19.878097,22.600023,19.696918,27.215816,30.534213,29.914326,29.931519
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,13.493244,25.448847,17.344519,28.968530,35.784723,25.448847,50.654657,28.968530,58.512845,65.208550,88.820203,74.027093
d8e66874-305e-4c80-9b75-5e764eb718ff,8.462962,25.907455,18.828896,26.649409,13.420956,25.907455,21.484991,26.649409,19.214737,29.642620,25.943683,30.647929
22f2d3ec-f7ea-4778-850d-bb111590202f,13.381107,26.531739,15.558408,23.538636,33.034891,26.531739,42.810114,23.538636,51.448794,63.013994,71.776199,54.761029
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,38.363388,68.866187,36.020241,68.278893,88.233697,68.866187,72.222070,68.278893,126.117879,129.653506,105.203776,128.933280


In [61]:
df_homegeneity_series.columns = ['homo1', 'homo2', 'homo3', 'homo4', 'homo5', 'homo6', 'homo7', 'homo8', 'homo9', 'homo10', 'homo11', 'homo12']

In [62]:
df_homegeneity_series.head()

,homo1,homo2,homo3,homo4,homo5,homo6,homo7,homo8,homo9,homo10,homo11,homo12
7be6b4de-afe9-43c0-a581-0f49608c8976,0.478469,0.425901,0.488767,0.428581,0.398995,0.425901,0.408006,0.428581,0.367263,0.369986,0.382874,0.374161
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,0.433721,0.353260,0.425887,0.350760,0.325174,0.353260,0.329670,0.350760,0.286510,0.279668,0.300036,0.276165
d8e66874-305e-4c80-9b75-5e764eb718ff,0.585204,0.511780,0.607219,0.510257,0.449638,0.511780,0.478615,0.510257,0.404295,0.417904,0.439191,0.417628
22f2d3ec-f7ea-4778-850d-bb111590202f,0.397197,0.330512,0.406560,0.338869,0.302493,0.330512,0.313650,0.338869,0.271871,0.268685,0.287263,0.279431
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,0.608064,0.524895,0.616607,0.524697,0.466775,0.524895,0.479737,0.524697,0.407190,0.411033,0.422789,0.410085


In [63]:
df_correlation_series.columns = ['cor1', 'cor2', 'cor3', 'cor4', 'cor5', 'cor6', 'cor7', 'cor8', 'cor9', 'cor10', 'cor11', 'cor12']

In [64]:
df_correlation_series.head()

,cor1,cor2,cor3,cor4,cor5,cor6,cor7,cor8,cor9,cor10,cor11,cor12
7be6b4de-afe9-43c0-a581-0f49608c8976,0.998827,0.998123,0.998880,0.998140,0.998068,0.998123,0.997864,0.998140,0.997432,0.997114,0.997171,0.997171
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,0.998589,0.997336,0.998186,0.996967,0.996257,0.997336,0.994695,0.996967,0.993877,0.993162,0.990687,0.992237
d8e66874-305e-4c80-9b75-5e764eb718ff,0.995714,0.986786,0.990493,0.986408,0.993174,0.986786,0.989128,0.986408,0.990185,0.984785,0.986846,0.984268
22f2d3ec-f7ea-4778-850d-bb111590202f,0.998548,0.997118,0.998311,0.997443,0.996412,0.997118,0.995349,0.997443,0.994409,0.993144,0.992196,0.994042
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,0.992852,0.987152,0.993285,0.987261,0.983551,0.987152,0.986507,0.987261,0.976478,0.975743,0.980305,0.975878


In [66]:
len(df_moments_series.columns)

24

In [69]:
df_moments_series.columns = ['m00', 'm10', 'm01', 'm20', 'm11', 'm02', 'm30', 'm21', 'm12', 'm03', 'mu20', 'mu11', 'mu02',
 'mu30', 'mu21', 'mu12', 'mu03', 'nu20', 'nu11', 'nu02', 'nu30', 'nu21', 'nu12', 'nu03']

In [70]:
df_moments_series.head()

,m00,m10,m01,m20,m11,m02,m30,m21,m12,m03,...,mu21,mu12,mu03,nu20,nu11,nu02,nu30,nu21,nu12,nu03
7be6b4de-afe9-43c0-a581-0f49608c8976,92240919.0,5.070445e+10,6.308803e+10,3.380680e+13,3.479384e+13,4.850783e+13,2.506429e+16,2.302899e+16,2.681212e+16,3.960552e+16,...,-2.191225e+14,-9.215689e+12,-9.017184e+14,0.000698,0.000013,0.000630,-5.361063e-07,-2.681501e-06,-1.127765e-07,-0.000011
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,118773763.0,5.702812e+10,6.882508e+10,3.506427e+13,3.283707e+13,5.003908e+13,2.427734e+16,1.974155e+16,2.386543e+16,3.964900e+16,...,-3.765553e+14,8.144301e+13,-1.118574e+15,0.000545,-0.000015,0.000720,4.159614e-07,-2.449218e-06,5.297275e-07,-0.000007
d8e66874-305e-4c80-9b75-5e764eb718ff,170334063.0,8.509673e+10,9.122167e+10,5.632774e+13,4.505956e+13,6.352576e+13,4.219903e+16,2.945350e+16,3.110337e+16,4.927270e+16,...,-1.993823e+14,-8.315358e+13,-4.637054e+14,0.000476,-0.000018,0.000506,6.742417e-07,-5.265419e-07,-2.195974e-07,-0.000001
22f2d3ec-f7ea-4778-850d-bb111590202f,140242338.0,6.231079e+10,7.799605e+10,3.665582e+13,3.418110e+13,5.462500e+13,2.488382e+16,1.960536e+16,2.372350e+16,4.229618e+16,...,-3.603867e+14,-2.051414e+13,-5.940782e+14,0.000456,-0.000024,0.000572,2.687185e-06,-1.547287e-06,-8.807556e-08,-0.000003
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,130111108.0,6.867372e+10,7.624595e+10,4.793634e+13,4.032555e+13,5.553744e+13,3.734241e+16,2.801977e+16,2.945962e+16,4.447764e+16,...,-1.581104e+14,5.004505e+13,-7.919957e+14,0.000691,0.000005,0.000641,-1.546897e-06,-8.187936e-07,2.591643e-07,-0.000004


In [71]:
len(df_hu_series.columns)

7

In [72]:
df_hu_series.columns = ['hu1', 'hu2', 'hu3', 'hu4', 'hu5', 'hu6', 'hu7']

In [73]:
df_hu_series.head()

,hu1,hu2,hu3,hu4,hu5,hu6,hu7
7be6b4de-afe9-43c0-a581-0f49608c8976,0.001327,5.307315e-09,8.980594e-12,1.885562e-10,7.592121e-21,-1.222616e-14,1.601241e-21
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,0.001265,3.164759e-08,1.381653e-12,9.546466e-11,2.493175e-22,1.697683e-14,1.067662e-21
d8e66874-305e-4c80-9b75-5e764eb718ff,0.000982,2.127836e-09,1.903034e-12,3.273141e-12,-6.970976e-24,1.409330e-16,-4.258934e-24
22f2d3ec-f7ea-4778-850d-bb111590202f,0.001028,1.571476e-08,1.308412e-11,2.354821e-11,-3.048701e-22,2.186856e-15,2.791155e-22
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,0.001332,2.515514e-09,8.109023e-12,2.586700e-11,-5.673887e-23,-9.863532e-16,3.703091e-22


In [74]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
7be6b4de-afe9-43c0-a581-0f49608c8976,7.36635,5.10596,"[92240919.0, 50704447353.0, 63088033389.0, 338...","[0.0013273545672697106, 5.307314779860795e-09,...",10718,0.712217,-2,0.825223,763.759,75,828.573,100.176,"[0.9988267630814048, 0.9981225489241269, 0.998...","[0.4784690393800168, 0.4259010474717295, 0.488...","[12.430608695320135, 19.878097023589405, 11.85...","[0.09403051421430261, 0.09128950002810914, 0.0..."
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,7.80413,5.40941,"[118773763.0, 57028122636.0, 68825079276.0, 35...","[0.0012646159119832335, 3.16475855769297e-08, ...",780,0.932381,-2,0.648379,309.907,16,83.0361,116.045,"[0.9985891998835106, 0.9973358959175828, 0.998...","[0.43372085077405403, 0.35326013611833573, 0.4...","[13.493244249450141, 25.448846615812855, 17.34...","[0.03328544296315335, 0.030327285573110596, 0...."
d8e66874-305e-4c80-9b75-5e764eb718ff,6.8283,4.73302,"[170334063.0, 85096727399.0, 91221672211.0, 56...","[0.0009818426448239955, 2.1278358422286364e-09...",912,0.935675,1,0.660391,200.018,17,98.6053,153.609,"[0.9957137994646331, 0.9867863821491449, 0.990...","[0.5852036010239059, 0.5117796098282055, 0.607...","[8.462962220491203, 25.907455025135455, 18.828...","[0.0510614483913255, 0.045511998024517275, 0.0..."
22f2d3ec-f7ea-4778-850d-bb111590202f,7.69593,5.33441,"[140242338.0, 62310789492.0, 77996053536.0, 36...","[0.0010279676066873146, 1.5714761019644863e-08...",341,0.879888,1,0.669941,100.142,7,34.5398,114.053,"[0.9985476796688767, 0.9971176416958397, 0.998...","[0.39719662527615573, 0.3305116441450096, 0.40...","[13.381106618096288, 26.531738728692645, 15.55...","[0.040514058469510505, 0.03853141907174066, 0...."
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,7.54643,5.23079,"[130111108.0, 68673721574.0, 76245952093.0, 47...","[0.001331840596963884, 2.515513779530961e-09, ...",568,0.915212,1,0.690158,260.332,42,123.288,112.061,"[0.9928517884301251, 0.9871516099477623, 0.993...","[0.608064329767518, 0.5248951911077396, 0.6166...","[38.36338816593354, 68.86618717684847, 36.0202...","[0.041865531996016635, 0.037771669350363746, 0..."


In [80]:
df.columns = ['s_entropy', 'entropy_simple', 'moments', 'hu_moments', 'max_area', 'eccentricity', 'euler_number', 'solidity', 'perimeter', 'mean_area', 'std_area', 'threshold', 'correlation', 'homogeneity', 'contrast', 'energy']

In [81]:
df.head()

,s_entropy,entropy_simple,moments,hu_moments,max_area,eccentricity,euler_number,solidity,perimeter,mean_area,std_area,threshold,correlation,homogeneity,contrast,energy
7be6b4de-afe9-43c0-a581-0f49608c8976,7.36635,5.10596,"[92240919.0, 50704447353.0, 63088033389.0, 338...","[0.0013273545672697106, 5.307314779860795e-09,...",10718,0.712217,-2,0.825223,763.759,75,828.573,100.176,"[0.9988267630814048, 0.9981225489241269, 0.998...","[0.4784690393800168, 0.4259010474717295, 0.488...","[12.430608695320135, 19.878097023589405, 11.85...","[0.09403051421430261, 0.09128950002810914, 0.0..."
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,7.80413,5.40941,"[118773763.0, 57028122636.0, 68825079276.0, 35...","[0.0012646159119832335, 3.16475855769297e-08, ...",780,0.932381,-2,0.648379,309.907,16,83.0361,116.045,"[0.9985891998835106, 0.9973358959175828, 0.998...","[0.43372085077405403, 0.35326013611833573, 0.4...","[13.493244249450141, 25.448846615812855, 17.34...","[0.03328544296315335, 0.030327285573110596, 0...."
d8e66874-305e-4c80-9b75-5e764eb718ff,6.8283,4.73302,"[170334063.0, 85096727399.0, 91221672211.0, 56...","[0.0009818426448239955, 2.1278358422286364e-09...",912,0.935675,1,0.660391,200.018,17,98.6053,153.609,"[0.9957137994646331, 0.9867863821491449, 0.990...","[0.5852036010239059, 0.5117796098282055, 0.607...","[8.462962220491203, 25.907455025135455, 18.828...","[0.0510614483913255, 0.045511998024517275, 0.0..."
22f2d3ec-f7ea-4778-850d-bb111590202f,7.69593,5.33441,"[140242338.0, 62310789492.0, 77996053536.0, 36...","[0.0010279676066873146, 1.5714761019644863e-08...",341,0.879888,1,0.669941,100.142,7,34.5398,114.053,"[0.9985476796688767, 0.9971176416958397, 0.998...","[0.39719662527615573, 0.3305116441450096, 0.40...","[13.381106618096288, 26.531738728692645, 15.55...","[0.040514058469510505, 0.03853141907174066, 0...."
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,7.54643,5.23079,"[130111108.0, 68673721574.0, 76245952093.0, 47...","[0.001331840596963884, 2.515513779530961e-09, ...",568,0.915212,1,0.690158,260.332,42,123.288,112.061,"[0.9928517884301251, 0.9871516099477623, 0.993...","[0.608064329767518, 0.5248951911077396, 0.6166...","[38.36338816593354, 68.86618717684847, 36.0202...","[0.041865531996016635, 0.037771669350363746, 0..."


In [82]:
d = df.drop(['moments'], axis=1)

In [83]:
d.head()

,s_entropy,entropy_simple,hu_moments,max_area,eccentricity,euler_number,solidity,perimeter,mean_area,std_area,threshold,correlation,homogeneity,contrast,energy
7be6b4de-afe9-43c0-a581-0f49608c8976,7.36635,5.10596,"[0.0013273545672697106, 5.307314779860795e-09,...",10718,0.712217,-2,0.825223,763.759,75,828.573,100.176,"[0.9988267630814048, 0.9981225489241269, 0.998...","[0.4784690393800168, 0.4259010474717295, 0.488...","[12.430608695320135, 19.878097023589405, 11.85...","[0.09403051421430261, 0.09128950002810914, 0.0..."
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,7.80413,5.40941,"[0.0012646159119832335, 3.16475855769297e-08, ...",780,0.932381,-2,0.648379,309.907,16,83.0361,116.045,"[0.9985891998835106, 0.9973358959175828, 0.998...","[0.43372085077405403, 0.35326013611833573, 0.4...","[13.493244249450141, 25.448846615812855, 17.34...","[0.03328544296315335, 0.030327285573110596, 0...."
d8e66874-305e-4c80-9b75-5e764eb718ff,6.8283,4.73302,"[0.0009818426448239955, 2.1278358422286364e-09...",912,0.935675,1,0.660391,200.018,17,98.6053,153.609,"[0.9957137994646331, 0.9867863821491449, 0.990...","[0.5852036010239059, 0.5117796098282055, 0.607...","[8.462962220491203, 25.907455025135455, 18.828...","[0.0510614483913255, 0.045511998024517275, 0.0..."
22f2d3ec-f7ea-4778-850d-bb111590202f,7.69593,5.33441,"[0.0010279676066873146, 1.5714761019644863e-08...",341,0.879888,1,0.669941,100.142,7,34.5398,114.053,"[0.9985476796688767, 0.9971176416958397, 0.998...","[0.39719662527615573, 0.3305116441450096, 0.40...","[13.381106618096288, 26.531738728692645, 15.55...","[0.040514058469510505, 0.03853141907174066, 0...."
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,7.54643,5.23079,"[0.001331840596963884, 2.515513779530961e-09, ...",568,0.915212,1,0.690158,260.332,42,123.288,112.061,"[0.9928517884301251, 0.9871516099477623, 0.993...","[0.608064329767518, 0.5248951911077396, 0.6166...","[38.36338816593354, 68.86618717684847, 36.0202...","[0.041865531996016635, 0.037771669350363746, 0..."


In [84]:

df = df.drop(['moments', 'hu_moments', 'correlation', 'homogeneity', 'contrast', 'energy'], axis=1)

In [85]:
df.head()

,s_entropy,entropy_simple,max_area,eccentricity,euler_number,solidity,perimeter,mean_area,std_area,threshold
7be6b4de-afe9-43c0-a581-0f49608c8976,7.36635,5.10596,10718,0.712217,-2,0.825223,763.759,75,828.573,100.176
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,7.80413,5.40941,780,0.932381,-2,0.648379,309.907,16,83.0361,116.045
d8e66874-305e-4c80-9b75-5e764eb718ff,6.8283,4.73302,912,0.935675,1,0.660391,200.018,17,98.6053,153.609
22f2d3ec-f7ea-4778-850d-bb111590202f,7.69593,5.33441,341,0.879888,1,0.669941,100.142,7,34.5398,114.053
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,7.54643,5.23079,568,0.915212,1,0.690158,260.332,42,123.288,112.061


In [86]:
df = df.merge(df_moments_series, left_index=True, right_index=True)

In [87]:
df.head()

,s_entropy,entropy_simple,max_area,eccentricity,euler_number,solidity,perimeter,mean_area,std_area,threshold,...,mu21,mu12,mu03,nu20,nu11,nu02,nu30,nu21,nu12,nu03
7be6b4de-afe9-43c0-a581-0f49608c8976,7.36635,5.10596,10718,0.712217,-2,0.825223,763.759,75,828.573,100.176,...,-2.191225e+14,-9.215689e+12,-9.017184e+14,0.000698,0.000013,0.000630,-5.361063e-07,-2.681501e-06,-1.127765e-07,-0.000011
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,7.80413,5.40941,780,0.932381,-2,0.648379,309.907,16,83.0361,116.045,...,-3.765553e+14,8.144301e+13,-1.118574e+15,0.000545,-0.000015,0.000720,4.159614e-07,-2.449218e-06,5.297275e-07,-0.000007
d8e66874-305e-4c80-9b75-5e764eb718ff,6.8283,4.73302,912,0.935675,1,0.660391,200.018,17,98.6053,153.609,...,-1.993823e+14,-8.315358e+13,-4.637054e+14,0.000476,-0.000018,0.000506,6.742417e-07,-5.265419e-07,-2.195974e-07,-0.000001
22f2d3ec-f7ea-4778-850d-bb111590202f,7.69593,5.33441,341,0.879888,1,0.669941,100.142,7,34.5398,114.053,...,-3.603867e+14,-2.051414e+13,-5.940782e+14,0.000456,-0.000024,0.000572,2.687185e-06,-1.547287e-06,-8.807556e-08,-0.000003
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,7.54643,5.23079,568,0.915212,1,0.690158,260.332,42,123.288,112.061,...,-1.581104e+14,5.004505e+13,-7.919957e+14,0.000691,0.000005,0.000641,-1.546897e-06,-8.187936e-07,2.591643e-07,-0.000004


In [88]:
df = df.merge(df_hu_series, left_index = True, right_index=True); df.head()

,s_entropy,entropy_simple,max_area,eccentricity,euler_number,solidity,perimeter,mean_area,std_area,threshold,...,nu21,nu12,nu03,hu1,hu2,hu3,hu4,hu5,hu6,hu7
7be6b4de-afe9-43c0-a581-0f49608c8976,7.36635,5.10596,10718,0.712217,-2,0.825223,763.759,75,828.573,100.176,...,-2.681501e-06,-1.127765e-07,-0.000011,0.001327,5.307315e-09,8.980594e-12,1.885562e-10,7.592121e-21,-1.222616e-14,1.601241e-21
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,7.80413,5.40941,780,0.932381,-2,0.648379,309.907,16,83.0361,116.045,...,-2.449218e-06,5.297275e-07,-0.000007,0.001265,3.164759e-08,1.381653e-12,9.546466e-11,2.493175e-22,1.697683e-14,1.067662e-21
d8e66874-305e-4c80-9b75-5e764eb718ff,6.8283,4.73302,912,0.935675,1,0.660391,200.018,17,98.6053,153.609,...,-5.265419e-07,-2.195974e-07,-0.000001,0.000982,2.127836e-09,1.903034e-12,3.273141e-12,-6.970976e-24,1.409330e-16,-4.258934e-24
22f2d3ec-f7ea-4778-850d-bb111590202f,7.69593,5.33441,341,0.879888,1,0.669941,100.142,7,34.5398,114.053,...,-1.547287e-06,-8.807556e-08,-0.000003,0.001028,1.571476e-08,1.308412e-11,2.354821e-11,-3.048701e-22,2.186856e-15,2.791155e-22
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,7.54643,5.23079,568,0.915212,1,0.690158,260.332,42,123.288,112.061,...,-8.187936e-07,2.591643e-07,-0.000004,0.001332,2.515514e-09,8.109023e-12,2.586700e-11,-5.673887e-23,-9.863532e-16,3.703091e-22


In [89]:
df = df.merge(df_correlation_series, left_index=True, right_index=True); df.head()

,s_entropy,entropy_simple,max_area,eccentricity,euler_number,solidity,perimeter,mean_area,std_area,threshold,...,cor3,cor4,cor5,cor6,cor7,cor8,cor9,cor10,cor11,cor12
7be6b4de-afe9-43c0-a581-0f49608c8976,7.36635,5.10596,10718,0.712217,-2,0.825223,763.759,75,828.573,100.176,...,0.998880,0.998140,0.998068,0.998123,0.997864,0.998140,0.997432,0.997114,0.997171,0.997171
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,7.80413,5.40941,780,0.932381,-2,0.648379,309.907,16,83.0361,116.045,...,0.998186,0.996967,0.996257,0.997336,0.994695,0.996967,0.993877,0.993162,0.990687,0.992237
d8e66874-305e-4c80-9b75-5e764eb718ff,6.8283,4.73302,912,0.935675,1,0.660391,200.018,17,98.6053,153.609,...,0.990493,0.986408,0.993174,0.986786,0.989128,0.986408,0.990185,0.984785,0.986846,0.984268
22f2d3ec-f7ea-4778-850d-bb111590202f,7.69593,5.33441,341,0.879888,1,0.669941,100.142,7,34.5398,114.053,...,0.998311,0.997443,0.996412,0.997118,0.995349,0.997443,0.994409,0.993144,0.992196,0.994042
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,7.54643,5.23079,568,0.915212,1,0.690158,260.332,42,123.288,112.061,...,0.993285,0.987261,0.983551,0.987152,0.986507,0.987261,0.976478,0.975743,0.980305,0.975878


In [90]:
df = df.merge(df_homegeneity_series, left_index=True, right_index=True); df.head()

,s_entropy,entropy_simple,max_area,eccentricity,euler_number,solidity,perimeter,mean_area,std_area,threshold,...,homo3,homo4,homo5,homo6,homo7,homo8,homo9,homo10,homo11,homo12
7be6b4de-afe9-43c0-a581-0f49608c8976,7.36635,5.10596,10718,0.712217,-2,0.825223,763.759,75,828.573,100.176,...,0.488767,0.428581,0.398995,0.425901,0.408006,0.428581,0.367263,0.369986,0.382874,0.374161
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,7.80413,5.40941,780,0.932381,-2,0.648379,309.907,16,83.0361,116.045,...,0.425887,0.350760,0.325174,0.353260,0.329670,0.350760,0.286510,0.279668,0.300036,0.276165
d8e66874-305e-4c80-9b75-5e764eb718ff,6.8283,4.73302,912,0.935675,1,0.660391,200.018,17,98.6053,153.609,...,0.607219,0.510257,0.449638,0.511780,0.478615,0.510257,0.404295,0.417904,0.439191,0.417628
22f2d3ec-f7ea-4778-850d-bb111590202f,7.69593,5.33441,341,0.879888,1,0.669941,100.142,7,34.5398,114.053,...,0.406560,0.338869,0.302493,0.330512,0.313650,0.338869,0.271871,0.268685,0.287263,0.279431
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,7.54643,5.23079,568,0.915212,1,0.690158,260.332,42,123.288,112.061,...,0.616607,0.524697,0.466775,0.524895,0.479737,0.524697,0.407190,0.411033,0.422789,0.410085


In [91]:
df = df.merge(df_contrast_series, left_index=True, right_index=True); df.head()

,s_entropy,entropy_simple,max_area,eccentricity,euler_number,solidity,perimeter,mean_area,std_area,threshold,...,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12
7be6b4de-afe9-43c0-a581-0f49608c8976,7.36635,5.10596,10718,0.712217,-2,0.825223,763.759,75,828.573,100.176,...,11.853243,19.696918,20.470722,19.878097,22.600023,19.696918,27.215816,30.534213,29.914326,29.931519
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,7.80413,5.40941,780,0.932381,-2,0.648379,309.907,16,83.0361,116.045,...,17.344519,28.968530,35.784723,25.448847,50.654657,28.968530,58.512845,65.208550,88.820203,74.027093
d8e66874-305e-4c80-9b75-5e764eb718ff,6.8283,4.73302,912,0.935675,1,0.660391,200.018,17,98.6053,153.609,...,18.828896,26.649409,13.420956,25.907455,21.484991,26.649409,19.214737,29.642620,25.943683,30.647929
22f2d3ec-f7ea-4778-850d-bb111590202f,7.69593,5.33441,341,0.879888,1,0.669941,100.142,7,34.5398,114.053,...,15.558408,23.538636,33.034891,26.531739,42.810114,23.538636,51.448794,63.013994,71.776199,54.761029
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,7.54643,5.23079,568,0.915212,1,0.690158,260.332,42,123.288,112.061,...,36.020241,68.278893,88.233697,68.866187,72.222070,68.278893,126.117879,129.653506,105.203776,128.933280


In [92]:
df = df.merge(df_energy_series, left_index=True, right_index=True); df.head()

,s_entropy,entropy_simple,max_area,eccentricity,euler_number,solidity,perimeter,mean_area,std_area,threshold,...,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12
7be6b4de-afe9-43c0-a581-0f49608c8976,7.36635,5.10596,10718,0.712217,-2,0.825223,763.759,75,828.573,100.176,...,0.093472,0.091521,0.091027,0.091290,0.090637,0.091521,0.089180,0.088050,0.088643,0.088414
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,7.80413,5.40941,780,0.932381,-2,0.648379,309.907,16,83.0361,116.045,...,0.034154,0.030249,0.029252,0.030327,0.030980,0.030249,0.027461,0.027149,0.029702,0.027061
d8e66874-305e-4c80-9b75-5e764eb718ff,6.8283,4.73302,912,0.935675,1,0.660391,200.018,17,98.6053,153.609,...,0.052772,0.045562,0.041105,0.045512,0.043130,0.045562,0.038051,0.039117,0.040497,0.039245
22f2d3ec-f7ea-4778-850d-bb111590202f,7.69593,5.33441,341,0.879888,1,0.669941,100.142,7,34.5398,114.053,...,0.041768,0.038920,0.037388,0.038531,0.039113,0.038920,0.035912,0.035952,0.037979,0.036583
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,7.54643,5.23079,568,0.915212,1,0.690158,260.332,42,123.288,112.061,...,0.043143,0.037772,0.034728,0.037772,0.036010,0.037772,0.032033,0.032476,0.033457,0.032452


In [95]:
df.shape

(500, 89)

In [96]:
df.to_pickle('dataframe-500')

In [98]:
df['class'] = dcm_target

In [99]:
df.shape

(500, 90)

In [100]:
df.head()

,s_entropy,entropy_simple,max_area,eccentricity,euler_number,solidity,perimeter,mean_area,std_area,threshold,...,e4,e5,e6,e7,e8,e9,e10,e11,e12,class
7be6b4de-afe9-43c0-a581-0f49608c8976,7.36635,5.10596,10718,0.712217,-2,0.825223,763.759,75,828.573,100.176,...,0.091521,0.091027,0.091290,0.090637,0.091521,0.089180,0.088050,0.088643,0.088414,0
2dcdd159-2889-48d3-a0ce-5c7b1086c49d,7.80413,5.40941,780,0.932381,-2,0.648379,309.907,16,83.0361,116.045,...,0.030249,0.029252,0.030327,0.030980,0.030249,0.027461,0.027149,0.029702,0.027061,0
d8e66874-305e-4c80-9b75-5e764eb718ff,6.8283,4.73302,912,0.935675,1,0.660391,200.018,17,98.6053,153.609,...,0.045562,0.041105,0.045512,0.043130,0.045562,0.038051,0.039117,0.040497,0.039245,0
22f2d3ec-f7ea-4778-850d-bb111590202f,7.69593,5.33441,341,0.879888,1,0.669941,100.142,7,34.5398,114.053,...,0.038920,0.037388,0.038531,0.039113,0.038920,0.035912,0.035952,0.037979,0.036583,1
cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,7.54643,5.23079,568,0.915212,1,0.690158,260.332,42,123.288,112.061,...,0.037772,0.034728,0.037772,0.036010,0.037772,0.032033,0.032476,0.033457,0.032452,0


In [101]:
df.to_pickle('dataframe_500')